# Aufgabe 1 - Prozessorientierte Layoutgestaltung

## Mathematisches Modell

**Zielfunktion**

\begin{equation}
	\min\ \ Z = \sum_{h=1}^{p}\sum_{\stackrel{i=1}{ i\ne h}}^{p}\sum_{j=1}^{p}\sum_{\stackrel{k=1}{k\ne j} }^{p} t_{hi} \cdot d_{jk} \cdot x_{hj} \cdot x_{ik}
\end{equation}

**unter den Nebenbedingungen**

\begin{align}
&&  \sum_{j=1}^{p}x_{hj} &= 1 && \forall h \\[5pt]
&&  \sum_{h=1}^{p}x_{hj} &= 1 && \forall j \\[10pt]
&& x_{hj} &\in \{0,1\} &&
\end{align}


## Aufgabe 1b)
 Lesen Sie die beiden Dateien `Distanz.csv` und `Transport.csv` ein.

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import csv

### Distanz.csv

\begin{equation}
    \begin{pmatrix}
    0 & 4 & 7 & 3 & 4 & 8 \\
    4 & 0 & 3 & 5 & 8 & 4 \\
    7 & 3 & 0 & 4 & 7 & 3 \\
    3 & 5 & 4 & 0 & 3 & 5 \\
    4 & 8 & 7 & 3 & 0 & 4 \\
    8 & 4 & 3 & 5 & 4 & 0 \\
    \end{pmatrix}
\end{equation}

In [ ]:
distance=[]
with open("Distanz.csv", encoding="utf-8") as csv_file:
     csv_reader = csv.reader(csv_file)
     for row in csv_reader:
          rowAsInt = [int(item) for item in row]
          distance.append(rowAsInt)

### Transport.csv
\begin{equation}
    \begin{pmatrix}
    0 & 3 & 6 & 9 & 8 & 3 \\
    3 & 0 & 2 & 1 & 3 & 2 \\
    6 & 2 & 0 & 4 & 0 & 1 \\
    9 & 1 & 4 & 0 & 3 & 4 \\
    8 & 3 & 0 & 3 & 0 & 5 \\
    3 & 2 & 1 & 4 & 5 & 0 \\
    \end{pmatrix}
\end{equation}

In [ ]:
transportAmount=[]
with open("Transport.csv", encoding="utf-8") as csv_file:
     csv_reader = csv.reader(csv_file)
     for row in csv_reader:
          rowAsInt = [int(item) for item in row]
          transportAmount.append(rowAsInt)

### Definition der Mengen

In [ ]:
numberOfMaschines = len(transportAmount) # oder händisch: 6
maschines = range(numberOfMaschines)
numberOfPlaces = len(distance) # oder händisch: 6
places = range(numberOfPlaces)

## Aufgabe 1c)
Initialisieren Sie ein Gurobi-Modell und fügen Sie diesem die erforderlichen Entscheidungsvariablen und die Zielfunktion hinzu.

### Initialisierung des Modells

In [ ]:
m = gp.Model()

### Initialisierung der Variablen

In [ ]:
x = m.addVars(numberOfMaschines, numberOfPlaces, vtype=GRB.BINARY, name="x")

#x = {}
#for i in maschines:
#    for j in places:
#        x[i,j] = m.addVar(vtype=GRB.BINARY, name="x"+str(i)+str(j))

### Definition der Zielfunktion

\begin{equation}
	\min\ \ Z = \sum_{h=1}^{p}\sum_{\stackrel{i=1}{ i\ne h}}^{p}\sum_{j=1}^{p}\sum_{\stackrel{k=1}{k\ne j} }^{p} t_{hi} \cdot d_{jk} \cdot x_{hj} \cdot x_{ik}
\end{equation}

In [ ]:
m.setObjective(
    gp.quicksum(
        gp.quicksum(
            gp.quicksum(
                gp.quicksum(
                    transportAmount[h][i] * distance[j][k] * x[h,j] * x[i,k] 
                for i in maschines if i != h)
            for h in maschines) 
        for k in places if k != j) 
    for j in places),
    GRB.MINIMIZE)

## Aufgabe 1d)
Ergänzen Sie das Modell mit den erforderlichen Nebenbedingungen und lösen Sie das Modell.

### Hinzufügen der Nebenbedingungen

In [ ]:
for h in maschines:
    m.addConstr(gp.quicksum(x[h,j] for j in places) == 1, "NB1." + str(h))

#m.addConstrs((x.sum(h,'*') == 1 for h in maschines), "NB1")

In [ ]:
for j in places:
    m.addConstr(gp.quicksum(x[h,j] for h in maschines) == 1, "NB." + str(j))

#m.addConstrs((x.sum('*',j) == 1 for j in places), "NB2")

### Optimierung

In [ ]:
m.optimize()

## Aufgabe 1e)
Lassen Sie sich die Maschinenzuordnung ausgeben.

Muster: `Maschine M... wird Standort ... zugeordnet.`

In [ ]:
print("\nMaschinenzuordnung:\n")

for h in maschines:
    for j in places:
        if x[h,j].X > 0:
            # Extra: Konvertiere Index zu Bezeichnung:
            placeJ = chr(ord('A') + j)
            #Alternativ einfach j ausgeben.
            print("Maschine M" + str(h), "wird Standort", placeJ, "zugeordnet.")

## Aufgabe 1f)
Lassen Sie sich zusätzlich den gesamten Materialfluss ausgeben und den Materialfluss
zwischen den einzelnen Maschinen.

Muster: `Zwischen M... und M... fließen ... ME über eine Distanz von ... EE.`

`Insgesamter Materialfluss: ... ME*EE.`

In [ ]:
print("\nMaterialfluss:\n")

for h in range(numberOfMaschines-1):
    ''' 
    Um beide Richtungen des Materialsflusses zusammenzufassen werden nur die Werte der oberen Dreiecksmatrix der Distanzen verwendet. 
    Also nur Distanzen bei denen die zweite Maschine einen höheren Index hat als die erste Maschine. 
    Dies ist zulässig, da die Distanzmatrix symmetrisch ist. 
    '''
    for i in range(h+1, numberOfMaschines): 
        if i != h:
            for j in places:
                for k in places:
                    if k != j and x[h,j].X * x[i,k].X > 0:
                        print("Zwischen M"+str(h),"und M"+str(i),"fließen", transportAmount[h][i]+transportAmount[i][h], "ME über eine Distanz von", distance[j][k],
                        "EE.")
print("Insgesamter Materialfluss:", m.getAttr(GRB.Attr.ObjVal), "ME*EE")

## Aufgabe 1g)
Schreiben Sie die Maschinenzuordnung und den Materialfluss in eine `.txt`-Datei.

In [ ]:
with open("Ergebnis.txt","w", encoding="utf-8") as f:
    f.write("Ergebnis der Layoutgestaltung\n")
    
    f.write("\nMaschinenzuordnung:\n")
    for h in maschines:
        for j in places:
            if x[h,j].X > 0:
                placeJ = chr(ord('A') + j)
                #Alternativ einfach j ausgeben.
                f.write("Maschine M" + str(h) + " wird Standort " + placeJ + " zugeordnet.\n")
    
    f.write("\nMaterialfluss:\n")
    for h in range(numberOfMaschines-1):
        for i in range(h+1, numberOfMaschines):
            if i != h:
                for j in places:
                    for k in places:
                        if k != j and x[h,j].X * x[i,k].X > 0:
                            f.write("Zwischen M" + str(h) + " und M" + str(i) + " fließen " + str(transportAmount[h][i]+transportAmount[i][h])
                            + " ME über eine Distanz von " + str(distance[j][k]) + " EE.\n")